# Dataset: Connect-Four dataset
# Focus task: Choice of the best model for Human Game Play Application

## Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from random import sample

## Loading the raw data

In [2]:
# Reading raw data from dat file
data = pd.read_table("/Datasets/connectfour/connectfour.data", sep=",")

#set column names for data
data.columns = ['a1','a2','a3','a4','a5','a6','b1','b2','b3','b4','b5','b6','c1','c2','c3','c4','c5','c6',\
                'd1','d2','d3','d4','d5','d6','e1','e2','e3','e4','e5','e6','f1','f2','f3','f4','f5','f6',\
                'g1','g2','g3','g4','g5','g6','Class']

#exploring the raw dataset
data.head()

a1 a2 a3 a4 a5 a6 b1 b2 b3 b4  ... f4 f5 f6 g1 g2 g3 g4 g5 g6 Class
0  b  b  b  b  b  b  b  b  b  b  ...  b  b  b  b  b  b  b  b  b   win
1  b  b  b  b  b  b  o  b  b  b  ...  b  b  b  b  b  b  b  b  b   win
2  b  b  b  b  b  b  b  b  b  b  ...  b  b  b  b  b  b  b  b  b   win
3  o  b  b  b  b  b  b  b  b  b  ...  b  b  b  b  b  b  b  b  b   win
4  b  b  b  b  b  b  b  b  b  b  ...  b  b  b  o  b  b  b  b  b   win

[5 rows x 43 columns]

#### We have typecasted the variables after exploring the dataset, since we discovered that the features x0,...,x8 were only taking 3 values(-1,0,1) and the target variable was taking only 9 values (0...8).

In [3]:
#basic information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67556 entries, 0 to 67555
Data columns (total 43 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   a1      67556 non-null  object
 1   a2      67556 non-null  object
 2   a3      67556 non-null  object
 3   a4      67556 non-null  object
 4   a5      67556 non-null  object
 5   a6      67556 non-null  object
 6   b1      67556 non-null  object
 7   b2      67556 non-null  object
 8   b3      67556 non-null  object
 9   b4      67556 non-null  object
 10  b5      67556 non-null  object
 11  b6      67556 non-null  object
 12  c1      67556 non-null  object
 13  c2      67556 non-null  object
 14  c3      67556 non-null  object
 15  c4      67556 non-null  object
 16  c5      67556 non-null  object
 17  c6      67556 non-null  object
 18  d1      67556 non-null  object
 19  d2      67556 non-null  object
 20  d3      67556 non-null  object
 21  d4      67556 non-null  object
 22  d5      67556 non-null

In [4]:
#checking number of records
data.shape

(67556, 43)

In [5]:
#create labelencoding instance
le = preprocessing.LabelEncoder()

#perform label encoding on entire dataset
data = data.apply(le.fit_transform)

#check new values for each column
data.head()

a1  a2  a3  a4  a5  a6  b1  b2  b3  b4  ...  f4  f5  f6  g1  g2  g3  g4  \
0   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
1   0   0   0   0   0   0   1   0   0   0  ...   0   0   0   0   0   0   0   
2   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
3   1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
4   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   1   0   0   0   

   g5  g6  Class  
0   0   0      2  
1   0   0      2  
2   0   0      2  
3   0   0      2  
4   0   0      2  

[5 rows x 43 columns]

In [6]:
#checking the features for occurance of each categorical value
data.apply(pd.Series.value_counts)

a1     a2     a3     a4     a5     a6     b1     b2     b3     b4  ...  \
0  24981  43384  55332  61615  65264  67039  21771  41179  54351  61205  ...   
1  20324  12024   6275   3090   1155    364  19896  13264   6847   3309  ...   
2  22251  12148   5949   2851   1137    153  25889  13113   6358   3042  ...   

      f4     f5     f6     g1     g2     g3     g4     g5     g6  Class  
0  64838  66818  67468  29728  48103  58868  64300  66709  67464   6449  
1   1490    447     70  18945   9896   4446   1785    533     74  16635  
2   1228    291     18  18883   9557   4242   1471    314     18  44472  

[3 rows x 43 columns]

In [7]:
#check value assigned to each target class
print(le.classes_)

['draw' 'loss' 'win']


## Random sampling and cross validation setup

In [8]:
#shuffling the records of the dataframe before splitting for train, test and validation
shuffled_data = data.sample(frac=1, random_state=80).reset_index().drop(['index'],axis = 1)
print(shuffled_data.head())

   a1  a2  a3  a4  a5  a6  b1  b2  b3  b4  ...  f4  f5  f6  g1  g2  g3  g4  \
0   1   1   2   2   0   0   2   2   1   0  ...   0   0   0   0   0   0   0   
1   1   0   0   0   0   0   0   0   0   0  ...   0   0   0   2   2   0   0   
2   2   2   0   0   0   0   2   1   1   0  ...   0   0   0   0   0   0   0   
3   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   1   1   0   0   
4   0   0   0   0   0   0   1   0   0   0  ...   0   0   0   0   0   0   0   

   g5  g6  Class  
0   0   0      0  
1   0   0      1  
2   0   0      1  
3   0   0      2  
4   0   0      2  

[5 rows x 43 columns]


In [9]:
#shuffling the records of the dataframe before splitting for train, test and validation
shuffled_data = data.sample(frac=1, random_state=80).reset_index().drop(['index'],axis = 1)
print(shuffled_data.head())
x = shuffled_data.iloc[:,0:42]
y = shuffled_data.iloc[:,42:]

   a1  a2  a3  a4  a5  a6  b1  b2  b3  b4  ...  f4  f5  f6  g1  g2  g3  g4  \
0   1   1   2   2   0   0   2   2   1   0  ...   0   0   0   0   0   0   0   
1   1   0   0   0   0   0   0   0   0   0  ...   0   0   0   2   2   0   0   
2   2   2   0   0   0   0   2   1   1   0  ...   0   0   0   0   0   0   0   
3   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   1   1   0   0   
4   0   0   0   0   0   0   1   0   0   0  ...   0   0   0   0   0   0   0   

   g5  g6  Class  
0   0   0      0  
1   0   0      1  
2   0   0      1  
3   0   0      2  
4   0   0      2  

[5 rows x 43 columns]


In [10]:
#set K value for cross validation 
cv = KFold(n_splits=10, random_state=70, shuffle=True)

#making the splits
kfold = cv.split(x, y)
scores = []

## Model training

## We are testing the model that performed the best on the tic-tac-toe datasets: MLP as well as Random forest and Naive Bayes classifiers

## KNN model

In [11]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(x, y) 

/home/jadhav.m/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:200: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(metric='euclidean')

In [53]:
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
for k, (train, test) in enumerate(kfold):
    knn.fit(x.iloc[train, :], y.iloc[train].to_numpy()) 
    predictions = knn.predict(x.iloc[test, :])
    score = knn.score(x.iloc[test, :], y.iloc[test])
    scores.append(score)
print(f'Accuracy: {np.mean(scores)}')

/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shap

Accuracy: 0.7394013556609383


In [71]:
X_test = x.iloc[70:90, :]
y_test = y.iloc[70:90, :]
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1Score = f1_score(y_test, y_pred, average='macro')
print(accuracy, precision, recall, f1Score)

0.7 0.5185185185185185 0.5185185185185185 0.5185185185185185


# MLP model

In [12]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
mlp = MLPClassifier(hidden_layer_sizes=(256,256,128), max_iter=500, activation = 'relu',solver='sgd', random_state=20)
mlp.fit(x, y) 

/home/jadhav.m/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(hidden_layer_sizes=(256, 256, 128), max_iter=500, random_state=20,
              solver='sgd')

In [58]:
for k, (train, test) in enumerate(kfold):
    mlp.fit(x.iloc[train, :], y.iloc[train].to_numpy()) 
    predictions = mlp.predict(x.iloc[test, :])
    score = mlp.score(x.iloc[test, :], y.iloc[test])
    scores.append(score)
print(f'Accuracy: {np.mean(scores)}')

/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and th

Accuracy: 0.8073882074940432


In [69]:
X_test = x.iloc[70:90, :]
y_test = y.iloc[70:90, :]
y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1Score = f1_score(y_test, y_pred, average='macro')

In [70]:
print(accuracy, precision, recall, f1Score)

0.95 0.9666666666666667 0.8333333333333334 0.8713450292397661


# Decision tree Model

In [13]:
bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=20, max_samples=0.7, max_features=0.8)
bagging.fit(x, y) 

/home/jadhav.m/.local/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_features=0.8,
                  max_samples=0.7, n_estimators=20)

In [66]:
for k, (train, test) in enumerate(kfold):
    bagging.fit(x.iloc[train, :], y.iloc[train].to_numpy()) 
    predictions = bagging.predict(x.iloc[test, :])
    score = bagging.score(x.iloc[test, :], y.iloc[test])
    scores.append(score)
print(f'Accuracy: {np.mean(scores)}')

/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of 

Accuracy: 0.8082919522367703


In [68]:
X_test = x.iloc[70:90, :]
y_test = y.iloc[70:90, :]
y_pred = bagging.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1Score = f1_score(y_test, y_pred, average='macro')
print(accuracy, precision, recall, f1Score)

0.8 0.75 0.7222222222222222 0.719047619047619


# Random Forest model

In [14]:
clf = RandomForestClassifier(n_estimators=25, max_depth=30,  min_samples_split=5, random_state=13)

# Training with k fold cross validation
for k, (train, test) in enumerate(kfold):
    clf.fit(x.iloc[train, :], y.iloc[train].to_numpy()) 
    predictions = clf.predict(x.iloc[test, :])
    score = clf.score(x.iloc[test, :], y.iloc[test])
    scores.append(score)
print(f'Accuracy: {np.mean(scores)}')

/scratch/local/48447213/ipykernel_116063/3430470828.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(x.iloc[train, :], y.iloc[train].to_numpy())
/scratch/local/48447213/ipykernel_116063/3430470828.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(x.iloc[train, :], y.iloc[train].to_numpy())
/scratch/local/48447213/ipykernel_116063/3430470828.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(x.iloc[train, :], y.iloc[train].to_numpy())
/scratch/local/48447213/ipykernel_116063/3430470828.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example u

Accuracy: 0.8086326664589395


In [76]:
X_test = x.iloc[70:90, :]
y_test = y.iloc[70:90, :]
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1Score = f1_score(y_test, y_pred, average='macro')
print(accuracy, precision, recall, f1Score)

0.75 0.5164835164835164 0.5555555555555555 0.5227272727272727


/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
ext = ExtraTreesClassifier(n_estimators=50, max_depth=30, min_samples_split=4, random_state=0)

# Training with k fold cross validation
for k, (train, test) in enumerate(kfold):
    ext.fit(x.iloc[train, :], y.iloc[train].to_numpy()) 
    predictions = ext.predict(x.iloc[test, :])
    score = ext.score(x.iloc[test, :], y.iloc[test])
    scores.append(score)
print(f'Accuracy: {np.mean(scores)}')

Accuracy: 0.8086326664589395


In [81]:
X_test = x.iloc[70:90, :]
y_test = y.iloc[70:90, :]
y_pred = ext.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1Score = f1_score(y_test, y_pred, average='macro')
print(accuracy, precision, recall, f1Score)

0.9 0.8555555555555555 0.9259259259259259 0.8741228070175439


In [23]:
model = AdaBoostClassifier(n_estimators=100)

# Training with k fold cross validation
for k, (train, test) in enumerate(kfold):
    model.fit(x.iloc[train, :], y.iloc[train].to_numpy()) 
    predictions = model.predict(x.iloc[test, :])
    score = model.score(x.iloc[test, :], y.iloc[test])
    scores.append(score)
print(f'Accuracy: {np.mean(scores)}')

Accuracy: 0.8086326664589395


In [85]:
X_test = x.iloc[70:90, :]
y_test = y.iloc[70:90, :]
y_pred = gnb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1Score = f1_score(y_test, y_pred, average='macro')
print(accuracy, precision, recall, f1Score)

0.6 0.5098039215686274 0.4444444444444444 0.3974358974358975


/apps/pytorch/1.10/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive Bayes Classifier

In [17]:
gnb = GaussianNB()
# Training with k fold cross validation
for k, (train, test) in enumerate(kfold):
    gnb.fit(x.iloc[train, :], y.iloc[train].to_numpy()) 
    predictions = gnb.predict(x.iloc[test, :])
    score = gnb.score(x.iloc[test, :], y.iloc[test])
    scores.append(score)
print(f'Accuracy: {np.mean(scores)}')

Accuracy: 0.8086326664589395


In [90]:
X_test = x.iloc[70:90, :]
y_test = y.iloc[70:90, :]
y_pred = gnb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1Score = f1_score(y_test, y_pred, average='macro')
print(accuracy, precision, recall, f1Score)

0.25 0.11904761904761905 0.1851851851851852 0.14492753623188406


### Comparative Performance of All models tested

| Classifier | Accuracy | Precision | Recall |
|------------|----------|-----------|--------|
| k-NN       |   0.82   |  0.77     |   0.70 |
| MLP       |  0.92        |    0.89       |    0.87    |
| Decision Tree     |     0.8     |     0.75      |   0.72     |
| Random Forest     |     0.75     |     0.51      |   0.55     |
| Extra Tree    |     0.9     |     0.85     |   0.92     |
| Adaboost    |     0.6     |     0.50      |   0.44     |
| Naive Bayes    |     0.25     |     0.11      |   0.18     |
